In [2]:
using Pkg
using Revise
Pkg.activate(dirname(dirname(@__DIR__)))
using MacroEnergy
using Gurobi
using Plots
using DataFrames, CSV
using BenchmarkTools
using JLD2, FileIO
using JuMP

  Activating project at `~/Documents/MacroEnergy.jl`


In [3]:
include("../helper_functions.jl")

get_system_results_no_co2_cap (generic function with 1 method)

In [4]:
# Run model for no CO2 cap
(system, model) = run_case(@__DIR__; optimizer=Gurobi.Optimizer)
case = "0pct_co2_reduction"
results_dir = joinpath(@__DIR__,  "results")
mkpath(results_dir)
save_results(results_dir, case, system)

###### ###### ######
Running case at /Users/al3792/Documents/MacroEnergy.jl/ExampleSystems/cement_retrofit_current


┌ Info: Loading system from /Users/al3792/Documents/MacroEnergy.jl/ExampleSystems/cement_retrofit_current/system_data.json
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_system.jl:18
┌ Info: Loading system data
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_system_data.jl:18
┌ Info: Done loading system data. It took 0.38 seconds
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_system_data.jl:26
┌ Info: Generating system
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/generate_system.jl:18
┌ Warning: WeightTotal not found in time_data.json
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_time_data.jl:81
┌ Info: Using PeriodLength as default value for WeightTotal
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_time_data.jl:82
┌ Info: Done generating system. It took 3.32 seconds
└ @ MacroEnergy /Users/al3792/Documents/MacroEnerg

Set parameter Username
Set parameter LicenseID to value 2637694
Academic license - for non-commercial use only - expires 2026-03-17
Set parameter BarConvTol to value 0.001
Set parameter Crossover to value 0
Set parameter Method to value 2
Set parameter Method to value 2
Set parameter Crossover to value 0
Set parameter BarConvTol to value 0.001
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[arm] - Darwin 24.2.0 24C101)

CPU model: Apple M4 Pro
Thread count: 14 physical cores, 14 logical processors, using up to 14 threads

Non-default parameters:
Method  2
BarConvTol  0.001
Crossover  0

Optimize a model with 867267 rows, 604486 columns and 2898522 nonzeros
Model fingerprint: 0x4a56369b
Coefficient statistics:
  Matrix range     [1e-03, 2e+04]
  Objective range  [1e-01, 2e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+04]
Presolve removed 447323 rows and 245847 columns
Presolve time: 0.93s
Presolved: 419944 rows, 358639 columns, 1916832 nonzeros
Ordering ti

┌ Info: Writing capacity results to /Users/al3792/Documents/MacroEnergy.jl/ExampleSystems/cement_retrofit_current/results/capacity.csv
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/write_outputs/capacity.jl:118
┌ Info: Writing costs to /Users/al3792/Documents/MacroEnergy.jl/ExampleSystems/cement_retrofit_current/results/costs.csv
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/write_outputs/costs.jl:45


No CO2 cap results
1×2 DataFrame
 Row │ objective_value  co2_emissions 
     │ Float64          Float64       
─────┼────────────────────────────────
   1 │       1.78889e9      2.29455e7

"/Users/al3792/Documents/MacroEnergy.jl/ExampleSystems/cement_retrofit_current/results/0pct_co2_reduction_flows.csv"

In [ ]:
# Run model for different CO2 caps, if you have a no_cap_co2_level

no_cap_co2_level = nothing
co2_reductions = [0] # 0 Must be the first value
model = nothing

for co2_reduction in co2_reductions
    # Load system
    case_path = @__DIR__
    println("###### ###### ######")
    println("Running case at $(case_path)")
    system = MacroEnergy.load_system(case_path)

    # Get the CO1 node
    co2_node = MacroEnergy.get_nodes_sametype(system.locations, CO2)[1]

    if co2_reduction == 0 # If there is no CO2 cap
        case = "0pct_co2_reduction"
        co2_node.constraints = AbstractTypeConstraint[] # Remove CO2 cap constraint
    else # If there is not a CO2 cap
        case = string(floor(Int, co2_reduction * 100)) * "pct_co2_reduction"
        co2_node.rhs_policy[CO2CapConstraint] = no_cap_co2_level * (1 - co2_reduction) # Set CO2 cap
        print(no_cap_co2_level * (1 - co2_reduction))
    end

    # Run model
    model = MacroEnergy.generate_model(system)
    MacroEnergy.set_optimizer(model, Gurobi.Optimizer);
    MacroEnergy.set_optimizer_attributes(model, "BarConvTol"=>1e-3,"Crossover" => 0, "Method" => 2)
    MacroEnergy.optimize!(model)

    # Save results
    results_dir = joinpath(case_path, "results")
    mkpath(results_dir)
    save_results(results_dir, case, system)
    # @save case * "_results.jld2" system_results_df capacity_results_df results_8760_df
    # println("wrote results")

    # Record the no_cap_co2_level for next iterations
    if co2_reduction == 0
        no_cap_co2_level = MacroEnergy.value(sum(co2_node.operation_expr[:emissions]))
    end
end